In [ ]:
# ============================================================
# 📌 Celda 1: Instalación y carga de librerías
# ============================================================
#!pip install scikit-learn matplotlib seaborn pandas


In [ ]:
# ============================================================
# 📌 Celda 2: Importar librerías
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# ============================================================
# 📌 Celda 3: Cargar dataset (Wine Quality - Vino Tinto)
# ============================================================
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine = pd.read_csv(url, sep=";")
wine["quality"].value_counts()



In [ ]:
wine.head()

In [ ]:
# Creamos variable binaria: calidad buena (>=6) vs mala (<6)
wine["quality_label"] = (wine["quality"] >= 6).astype(int)

X = wine.drop(columns=["quality", "quality_label"])
y = wine["quality_label"]

In [ ]:
print("Dimensiones de X:", X.shape)
print("Distribución de clases:\n", y.value_counts(normalize=True))

wine.head()

In [ ]:
# ============================================================
# 📌 Celda 4: Dividir datos en entrenamiento y prueba
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Tamaño entrenamiento:", X_train.shape)
print("Tamaño prueba:", X_test.shape)


In [ ]:
# ============================================================
# 📌 Celda 5: Árbol "grande" con Gini (sin poda)
# ============================================================
clf_big_gini = DecisionTreeClassifier(criterion="gini", random_state=42)
clf_big_gini.fit(X_train, y_train)

y_train_pred = clf_big_gini.predict(X_train)
y_test_pred = clf_big_gini.predict(X_test)

print("🔹 Árbol grande - Gini (sin poda)")
print("Accuracy Entrenamiento:", accuracy_score(y_train, y_train_pred))
print("Accuracy Prueba:", accuracy_score(y_test, y_test_pred))



In [ ]:
# 📌 Reporte de métricas en el set de prueba
print("\nReporte en test:\n", classification_report(y_test, y_test_pred, target_names=["Mala","Buena"]))

# 📌 Matriz de confusión
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, cmap="Blues",
            xticklabels=["Mala","Buena"], yticklabels=["Mala","Buena"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión - Árbol grande (Gini)")
plt.show()

In [ ]:
# 📌 Visualización del árbol completo
plt.figure(figsize=(20,10))
plot_tree(clf_big_gini, filled=True, feature_names=X.columns, class_names=["Mala","Buena"], fontsize=8)
plt.title("Árbol grande (sin poda, criterio Gini)")
plt.show()

In [ ]:
# Ejemplo de interpretación de un nodo del árbol:
#
# alcohol <= 10.525
# 👉 La característica usada para dividir en este nodo es "alcohol".
#    Si el valor de alcohol es <= 10.525, el caso va a la rama izquierda.
#    Si es mayor a 10.525, el caso va a la rama derecha.
#
# gini = 0.498
# 👉 Impureza Gini del nodo.
#    El valor va de 0 (puro, todas las muestras de la misma clase) a 0.5 (mezcla máxima en binario).
#    Aquí 0.498 indica que las clases están casi balanceadas (mitad y mitad).
#
# samples = 1119
# 👉 Número total de muestras (filas del dataset) que llegan a este nodo.
#
# value = [521.0, 598.0]
# 👉 Distribución de clases en este nodo:
#    - 521 vinos son de la clase "Mala" (0).
#    - 598 vinos son de la clase "Buena" (1).
#
# class = Buena
# 👉 Clase mayoritaria en este nodo.
#    Como 598 > 521, la predicción por defecto en este nodo sería "Buena".


In [ ]:
# ============================================================
# 📌 Celda 6: Poda con max_depth
# ============================================================
clf_pruned = DecisionTreeClassifier(criterion="gini", max_depth=5, random_state=42)
clf_pruned.fit(X_train, y_train)

y_train_pred_pruned = clf_pruned.predict(X_train)
y_test_pred_pruned = clf_pruned.predict(X_test)

print("🔹 Árbol podado - Gini (max_depth=5)")
print("Accuracy Entrenamiento:", accuracy_score(y_train, y_train_pred_pruned))
print("Accuracy Prueba:", accuracy_score(y_test, y_test_pred_pruned))

# 📌 Visualización del árbol podado
plt.figure(figsize=(20,10))
plot_tree(clf_pruned, filled=True, feature_names=X.columns, class_names=["Mala","Buena"], fontsize=10)
plt.title("Árbol podado (max_depth=5, criterio Gini)")
plt.show()



In [ ]:
# ============================================================
# 📌 Celda 7: Cambiar criterio a "Entropía"
# ============================================================
clf_entropy = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
clf_entropy.fit(X_train, y_train)

y_pred_entropy = clf_entropy.predict(X_test)

print("🔹 Árbol podado - Entropía (max_depth=5)")
print("Accuracy en test:", accuracy_score(y_test, y_pred_entropy))

# 📌 Visualización del árbol con entropía
plt.figure(figsize=(20,10))
plot_tree(clf_entropy, filled=True, feature_names=X.columns, class_names=["Mala","Buena"], fontsize=10)
plt.title("Árbol podado (max_depth=5, criterio Entropía)")
plt.show()

# 📌 Explicación:
# - El valor por defecto en DecisionTreeClassifier es "gini".
# - Gini mide la probabilidad de clasificar mal un ejemplo si se elige aleatoriamente según la distribución de clases.
# - Entropía mide la incertidumbre (teoría de la información).
# - Efectos:
#   Gini → más rápido de calcular, favorece clases mayoritarias.
#   Entropía → puede equilibrar mejor divisiones si hay varias clases.
# - En la práctica, la diferencia de rendimiento suele ser pequeña.



In [ ]:
# ============================================================
# 📌 Celda 8: Búsqueda sistemática de hiperparámetros con GridSearchCV
# ============================================================
from sklearn.model_selection import GridSearchCV

# Definimos los hiperparámetros a explorar
param_grid = {
    "criterion": ["gini", "entropy"],      # función de impureza
    "max_depth": [3, 5, 7, None],          # profundidad máxima
    "min_samples_split": [2, 10, 20],      # mínimo de muestras para dividir
    "min_samples_leaf": [1, 5, 10],        # mínimo de muestras por hoja
    "max_features": [None, "sqrt", "log2"] # número máximo de variables a considerar en cada split
}

# Configuramos el GridSearch
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,                 # validación cruzada 5-fold
    scoring="accuracy",   # métrica a optimizar
    n_jobs=-1             # usa todos los cores disponibles
)

# Ajustamos el modelo
grid_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_tree = grid_search.best_estimator_

print("🔹 Mejor combinación de hiperparámetros:", grid_search.best_params_)
print("🔹 Mejor accuracy en validación cruzada:", grid_search.best_score_)





In [ ]:
# Evaluamos en el set de test
y_pred_best = best_tree.predict(X_test)

print("\nAccuracy en test:", accuracy_score(y_test, y_pred_best))
print("\nReporte en test:\n", classification_report(y_test, y_pred_best, target_names=["Mala","Buena"]))

# 📌 Matriz de confusión
sns.heatmap(confusion_matrix(y_test, y_pred_best), annot=True, cmap="Purples",
            xticklabels=["Mala","Buena"], yticklabels=["Mala","Buena"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión - Árbol optimizado con GridSearchCV")
plt.show()



In [ ]:
# 📌 Visualización del mejor árbol
plt.figure(figsize=(20,10))
plot_tree(best_tree, filled=True, feature_names=X.columns, class_names=["Mala","Buena"], fontsize=9)
plt.title("Árbol optimizado con GridSearchCV")
plt.show()

In [ ]:
# ============================================================
# 📌 Celda 9: Comparación final de modelos
# ============================================================
acc_results = {
    "Árbol grande (Gini)": accuracy_score(y_test, y_test_pred),
    "Árbol podado (Gini)": accuracy_score(y_test, y_test_pred_pruned),
    "Árbol podado (Entropía)": accuracy_score(y_test, y_pred_entropy),
    "Árbol optimizado (GridSearchCV)": accuracy_score(y_test, y_pred_best),
}

for modelo, acc in acc_results.items():
    print(f"{modelo}: {acc:.4f}")



In [ ]:
# ============================================================
# 📌 Celda 10: Comparación Train vs Test Accuracy
# ============================================================

# Accuracy en entrenamiento
train_acc_results = {
    "Árbol grande (Gini)": accuracy_score(y_train, y_train_pred),
    "Árbol podado (Gini)": accuracy_score(y_train, y_train_pred_pruned),
    "Árbol podado (Entropía)": accuracy_score(y_train, clf_entropy.predict(X_train)),
    "Árbol optimizado (GridSearchCV)": accuracy_score(y_train, best_tree.predict(X_train)),
}

# Accuracy en prueba (ya calculado en Celda 9)
test_acc_results = acc_results

# Convertimos a DataFrame para graficar
df_acc = pd.DataFrame({
    "Train": train_acc_results,
    "Test": test_acc_results
}).T  # transponemos para que los modelos queden como filas

# 📊 Gráfico comparativo con etiquetas
ax = df_acc.plot(kind="bar", figsize=(10,6), rot=15, color=["skyblue","salmon"])
plt.ylabel("Accuracy")
plt.title("Comparación Accuracy en Train vs Test - Árboles de Decisión")
plt.ylim(0,1)
plt.legend(title="Conjunto")

# Añadimos etiquetas numéricas encima de cada barra
for p in ax.patches:
    ax.annotate(
        f"{p.get_height():.2f}",             # texto con 2 decimales
        (p.get_x() + p.get_width() / 2, p.get_height()), # posición
        ha="center", va="bottom", fontsize=9, color="black", xytext=(0,3), textcoords="offset points"
    )

plt.show()



## 📌 Conclusión

Aunque el **árbol optimizado con GridSearchCV** muestra el **menor accuracy en el set de prueba**, es el modelo más recomendable porque alcanza un mejor equilibrio entre **complejidad y capacidad de generalización**.  
El **árbol grande con Gini** logra la mayor exactitud, pero lo hace a costa de un **sobreajuste evidente**: memoriza en exceso el conjunto de entrenamiento y pierde robustez ante datos nuevos.  

En cambio, el **árbol optimizado** ha sido ajustado mediante **validación cruzada**, lo que asegura que su desempeño no dependa de un único set de datos, sino que refleje un comportamiento más **estable, consistente e interpretable**.  
En aplicaciones reales, donde los datos futuros nunca son idénticos a los de entrenamiento, es preferible un modelo con un accuracy ligeramente menor, pero que sea mucho más **confiable y generalizable**.
